In [1]:
from utils import *
import os

In [2]:
os.getcwd()

'/root/autodl-tmp/ML_Script/3_Model'

In [3]:
os.chdir("../2_Data")

In [4]:
df_no = pd.read_csv("No_self.csv")
df_ex = pd.read_csv("Explicit_self.csv")
df_im = pd.read_csv("Implicit_self.csv")
df_exp5 = pd.read_csv("Exp5_all.csv")

In [5]:
df_Exp5_Morality = df_exp5.query('taskType == "Morality"')
df_Exp5_Emotion = df_exp5.query('taskType == "Emotion"')
df_Exp5_Person = df_exp5.query('taskType == "Person"')
df_Exp5_Scene = df_exp5.query('taskType == "Scene"')

df_ex_other = df_ex.query('Identity == "Other"')
df_ex_self = df_ex.query('Identity == "Self"')
df_im_other = df_im.query('Identity == "Other"')
df_im_self = df_im.query('Identity == "Self"')

In [6]:
y_no = df_no["Valence"].values
y_Exp5_Morality = df_Exp5_Morality["Valence"].values
y_Exp5_Emotion = df_Exp5_Emotion["Valence"].values
y_Exp5_Person = df_Exp5_Person["Valence"].values
y_Exp5_Scene = df_Exp5_Scene["Valence"].values

y_ex_other = df_ex_other["Valence"].values
y_ex_self = df_ex_self["Valence"].values

y_im_other = df_im_other["Valence"].values
y_im_self = df_im_self["Valence"].values

In [7]:
X_no = df_no.iloc[:, 4:7].values
X_Exp5_Morality = df_Exp5_Morality.iloc[:, 4:7].values
X_Exp5_Emotion = df_Exp5_Emotion.iloc[:, 4:7].values
X_Exp5_Person = df_Exp5_Person.iloc[:, 4:7].values
X_Exp5_Scene = df_Exp5_Scene.iloc[:, 4:7].values

X_ex_other = df_ex_other.iloc[:, 4:7].values
X_ex_self = df_ex_self.iloc[:, 4:7].values

X_im_other = df_im_other.iloc[:, 4:7].values
X_im_self = df_im_self.iloc[:, 4:7].values

In [8]:
groups_no = df_no["Subject"].values
groups_Exp5_Morality = df_Exp5_Morality["Subject"].values
groups_Exp5_Emotion = df_Exp5_Emotion["Subject"].values
groups_Exp5_Person = df_Exp5_Person["Subject"].values
groups_Exp5_Scene = df_Exp5_Scene["Subject"].values

groups_ex_other = df_ex_other["Subject"].values
groups_ex_self = df_ex_self["Subject"].values

groups_im_other = df_im_other["Subject"].values
groups_im_self = df_im_self["Subject"].values

In [9]:
names = ['No_Ref', 'Exp5_Morality', 'Exp5_Emotion', 'Exp5_Person', 'Exp5_Scene', 'Explicit_Other', 'Explicit_Self','Implicit_Other', "Implicit_Self"]
names4cross = ['No_Ref', 'Exp5_Morality', 'Exp5_Emotion', 'Exp5_Person', 'Exp5_Scene', 'Explicit_Other', 'Explicit_Self','Implicit_Other', "Implicit_Self"]
features = [X_no, X_Exp5_Morality, X_Exp5_Emotion, X_Exp5_Person, X_Exp5_Scene, X_ex_other, X_ex_self, X_im_other, X_im_self]
labels= [y_no, y_Exp5_Morality, y_Exp5_Emotion, y_Exp5_Person, y_Exp5_Scene, y_ex_other, y_ex_self, y_im_other, y_im_self]
groups = [groups_no, groups_Exp5_Morality, groups_Exp5_Emotion, groups_Exp5_Person, groups_Exp5_Scene, groups_ex_other, groups_ex_self, groups_im_other, groups_im_self]

In [ ]:
df_within_score = pd.DataFrame(None, columns=["subID", "score", "source", "target", "source_Identity","target_Identity", "source_ref", "target_ref"])
for i in range(len(names)):
    
    df_result, _, _ = lr_within_task(X=features[i], y=labels[i], group=groups[i], source=names[i])
    if np.unique(df_result['source']) in ['No_Ref', 'Exp5_Morality', 'Exp5_Emotion', 'Exp5_Person', 'Exp5_Scene']:
        df_result["source_ref"] = "None"
        df_result["source_Identity"] = "None"
    elif np.unique(df_result['source']) in ['Explicit_Other', 'Explicit_Self']:
        df_result["source_ref"] = "Explicit"
    elif np.unique(df_result['source']) in ['Implicit_Other', "Implicit_Self"]:
        df_result["source_ref"] = "Implicit"  

    if np.unique(df_result['source']) in ['Explicit_Other', 'Implicit_Other']:
        df_result["source_Identity"] = "Other"
    elif np.unique(df_result['source']) in ['Explicit_Self', "Implicit_Self"]:
        df_result["source_Identity"] = "Self"
    df_result["target_Identity"] = df_result.source_Identity
    df_result["target_ref"] = df_result.source_ref
    df_result = df_result[["subID", "score", "source", "target", "source_Identity","target_Identity", "source_ref", "target_ref"]]
    df_within_score = pd.concat([df_within_score, df_result])

In [ ]:
df_within_score.to_csv(os.path.join("../4_Result/", "within_score.csv"))

In [ ]:
df_cross_score = pd.DataFrame(None, columns=["subID", "score", "source", "target", "source_Identity","target_Identity", "source_ref", "target_ref"])
for i in range(len(names)):
    for j in range(len(names4cross)):
        if names[i] == names4cross[j]:
            pass
        else:
            df_result,_,_ = lr_cross_task2(X_source=features[i], y_source=labels[i], X_target=features[j], y_target=labels[j], source_group=groups[i], target_group=groups[j], source_name=names[i], target_name=names[j])
            
            if np.unique(df_result['source']) in ['No_Ref', 'Exp5_Morality', 'Exp5_Emotion', 'Exp5_Person', 'Exp5_Scene']:
                df_result["source_ref"] = "None"
                df_result["source_Identity"] = "None"
            elif np.unique(df_result['source']) in ['Explicit_Other', 'Explicit_Self']:
                df_result["source_ref"] = "Explicit"
            elif np.unique(df_result['source']) in ['Implicit_Other', "Implicit_Self"]:
                df_result["source_ref"] = "Implicit"  

            if np.unique(df_result['source']) in ['Explicit_Other', 'Implicit_Other']:
                df_result["source_Identity"] = "Other"
            elif np.unique(df_result['source']) in ['Explicit_Self', "Implicit_Self"]:
                df_result["source_Identity"] = "Self"

            if np.unique(df_result['target']) in ['No_Ref', 'Exp5_Morality', 'Exp5_Emotion', 'Exp5_Person', 'Exp5_Scene']:
                df_result["target_ref"] = "None"
                df_result["target_Identity"] = "None"
            elif np.unique(df_result['target']) in ['Explicit_Other', 'Explicit_Self']:
                df_result["target_ref"] = "Explicit"
            elif np.unique(df_result['target']) in ['Implicit_Other', "Implicit_Self"]:
                df_result["target_ref"] = "Implicit"  

            if np.unique(df_result['target']) in ['Explicit_Other', 'Implicit_Other']:
                df_result["target_Identity"] = "Other"
            elif np.unique(df_result['target']) in ['Explicit_Self', "Implicit_Self"]:
                df_result["target_Identity"] = "Self"

            df_result = df_result[["subID", "score", "source", "target", "source_Identity","target_Identity", "source_ref", "target_ref"]]

            df_cross_score = pd.concat([df_cross_score, df_result])

In [ ]:
df_cross_score.to_csv(os.path.join("../4_Result/", "cross_score.csv"))

In [ ]:
!/usr/bin/shutdown